In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, AveragePooling2D
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Flatten, Dense, GlobalAveragePooling2D
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize Data
def normalize(X_train, X_test):
    # 計算(50000, 32, 32, 3)四個軸向的平均值
    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    # 計算(50000, 32, 32, 3)四個軸向的標準差
    std = np.std(X_train, axis=(0, 1, 2, 3))
    # normalization standard score(Xn = (輸入 - 平均數) / 標準差)
    X_train = (X_train - mean) / (std + 1e-7)
    X_test = (X_test - mean) / (std + 1e-7)
    return X_train, X_test, mean, std

# Normalize Training and Testset
x_train, x_test, mean_train, std_train = normalize(x_train, x_test)


## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0], label=4,變成[0,0,0,0,1,0,0,0,0,0]
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()


classifier = Sequential()

input_img = (32, 32, 3)
classifier.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", input_shape=input_img, activation="relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", input_shape=input_img, activation="relu"))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Flatten())
classifier.add(Dense(units=100, activation="relu"))
classifier.add(Dense(units=10, activation="softmax"))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = classifier.fit(x_train, y_train, batch_size=200, epochs=10)
print(history.history)

# input_example=(np.zeros(shape=(1, 32, 32, 3))-mean_train)/(std_train+1e-7)
# classifier.predict(input_example)

import matplotlib.pyplot as plt
plt.plot(history.history["loss"])
plt.show()